In [2]:
import shapefile
import numpy as np
import sys,os
import h5py
from matplotlib import pyplot as plt
from matplotlib import collections

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats-mesh/tools/meshing_ats/meshing_ats')
#*sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats','meshing_ats'))
#*sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/meshing_ats/meshing_ats')
sys.path.append('/Users/ajc/Core/PyScript')
import meshing_ats
eps = 1.0

In [3]:
def dist(p0,p1):
    return np.linalg.norm(p0.x - p1.x)

def is_equal(p0,p1,eps=1.0):
    if (dist(p0,p1) < eps):
        return True
    return False

def centroid(points):
    return Point(sum(p.x for p in points)/len(points))

def ccw(A, B, C):
    """Tests whether the turn formed by A, B, and C is ccw"""
    return (B.x[0] - A.x[0]) * (C.x[1] - A.x[1]) > (B.x[1] - A.x[1]) * (C.x[0] - A.x[0])

def convex(points):
    """Returns 1 if convex, ccw ordered, -1 if convex but cw ordered, 0 if not convex"""
    is_ccw = []
    for i in range(len(points)):
        # Check every triplet of points
        A = points[i % len(points)]
        B = points[(i + 1) % len(points)]
        C = points[(i + 2) % len(points)]
        is_ccw.append(ccw(A,B,C))
    if all(is_ccw):
        return 1
    elif all([not(c) for c in is_ccw]):
        return -1
    return 0    

def star_convex(points):
    """Returns 1 if star-convex, ccw ordered, -1 if star-convex but cw ordered, 0 if not star-convex"""
    cent = centroid(points)
    convex_tris = []
    for i in range(len(points)):
        tri_points = [points[i], points[(i+1)%len(points)], cent]
        convex_tris.append(convex(tri_points))
    if all([cv_tri == 1 for cv_tri in convex_tris]):
        return 1
    elif all([cv_tri == -1 for cv_tri in convex_tris]):
        return -1
    return 0    

class Point(object):
    def __init__(self, x, decimals=1):
        self.x = np.array(x)
        self.xr = np.array([np.round(self.x[0],decimals), 
                            np.round(self.x[1],decimals)])
        
    def __hash__(self):
        return hash(tuple(self.xr))
        
    def __eq__(self, other):
        return is_equal(self, other, eps)
    
    def __repr__(self):
        return "Point: (%12.10g,%12.10g)"%(self.xr[0],self.xr[1])




In [4]:
# read the polygons via shapefile
sf_poly = shapefile.Reader("./Polygon_mesh_final/Polygon_mesh_final")
polygons = sf_poly.shapes()

nodes = []
conn = []

for s in polygons:
    poly_conn = []
    for p in s.points:
        myp = Point(p)
        if len(nodes) > 0:
            i = np.argmin(np.array([dist(myp, lcv) for lcv in nodes]))

            min_dist = dist(myp, nodes[i])
            if min_dist > eps:
                poly_conn.append(len(nodes))
                nodes.append(myp)
            else:
                poly_conn.append(i)
        else:
            nodes.append(myp)
            poly_conn.append(0)
    conn.append(poly_conn)
    
print (len(nodes))

xyz_nodes = np.zeros((len(nodes),3),'d')
for i,n in enumerate(nodes):
    xyz_nodes[i,0:2] = n.x

959


In [ ]:
# write the nodes file
with open("./Polygon_mesh_nodes_final.csv",'w') as fid:
    fid.write("point ID,X,Y\n")
    for i,n in enumerate(nodes):
        fid.write("%d,%16.16g,%16.16g\n"%(i,n.x[0],n.x[1]))


In [ ]:
# read the resulting shapefile with x,y,z nodes
sf_points = shapefile.Reader("./Polygon_mesh_nodes_sample_final/Polygon_mesh_nodes_sample_final")
shapes = sf_points.shapes()

xyz_points = np.array([[p.points[0][0], p.points[0][1], p.z[0]] for p in shapes])
points = [Point(xyz_points[i,0:2]) for i in range(xyz_points.shape[0])]

# match nodes to xyz_points
for i,n in enumerate(nodes):
    j_closest = np.argmin(np.array([dist(n,p) for p in points]))
    xyz_nodes[i,2] = xyz_points[j_closest,2]


In [ ]:
# potentially strip duplicate points
for c in conn:
    if c[0] == c[-1]:
        c.pop()

In [ ]:
%matplotlib 
# make the 2D mesh

m2 = meshing_ats.Mesh2D(xyz_nodes,conn)
m2.plot(color=['b','r','g','m','c','y','gray'])
plt.show()

In [ ]:
# check for non-star-convex cells
non_star_convex = []
for i,c in enumerate(conn):
    points = [nodes[j] for j in c]
    cv = star_convex(points)
    if cv == -1:
        c.reverse()
    elif cv == 1:
        # pass
        0 == 0
    else:
        print ("Non-star-convex cell:", i, c)
        plt.figure()
        c2 = c[:]
        c2.append(c2[0])
        coords = xyz_nodes[c2]
        plt.plot(coords[:,0],coords[:,1], 'b-x')
        cent = centroid(points)
        plt.scatter([cent.x[0],], [cent.x[1],], s=50, marker='s', c='r')
        non_star_convex.append(i)

fig,ax = plt.subplots(1,1)

m2.plot(color=['b','r','g','m','c'],ax=ax)

verts = [[xyz_nodes[i,0:2] for i in conn[p]] for p in non_star_convex]
gons = collections.PolyCollection(verts, facecolors='r')
ax.add_collection(gons)

plt.show()
print (gons)

In [ ]:
#Use VisIt to get IDs of High-centered polygons, the GIDs should be obtained in parallel runs
path = '/Users/ajc/Core/PreProcessing/PyScript/meshes/watershed-barrow/'
sys.path.append('')
def read_poly_ids():
    ids = []
    with open(os.path.join(path,'poly_char_gids/manually_char_polygons_new.txt')) as f:
        for line in f:
            if 'Zone:' in line:
                c = int(line.split()[1])
                if not c in ids:
                    ids.append(c)
    return ids


def get_ponded_depth():
    d1 = h5py.File('/Users/ajc/Projects/ATS-Data/OR-CONDO/ats-intermediate-ngee/surface_only/Rain_R1-serial/visdump_surface_data.h5','r')
    var = 'surface-ponded_depth.cell.0'
    keys = np.array(d1[var].keys(), 'i')
    keys = np.sort(keys, axis=None)
    cycle = '%s'%keys[-1]
    Ids_pd = d1[var][cycle][:]
    
    return Ids_pd
    

In [ ]:
#IDS1= read_poly_ids()

In [ ]:
IDS2= read_poly_ids()
#print (IDS2)

In [ ]:
#id1=np.sort(IDS1)
hcp_gids=np.sort(IDS2)
#print (id1, hcp_gids)
print (len(hcp_gids),hcp_gids)

In [ ]:
def irz_layer_thickness_gids(id,d_thick):
    if id in hcp_gids:
        thickness = d_thick #0.4
    else:
        thickness= 0.0
    return thickness

sampl = np.random.uniform(low=20, high=40, size=(468,))

sampl = [np.ceil(x) for x in sampl]
irz_rand = [round(0.01 * x,4) if x %2 ==0 else round(0.01 * (x-1),4) for x in sampl]
print (irz_rand)

In [ ]:
#Ids = read_poly_ids()
#ponded_depth = get_ponded_depth()
gid_based = True

peat_thickness = np.zeros(m2.num_cells())
irz_thickness = np.zeros(m2.num_cells())
hcp_irz = 0.5
lcp_irz = 0.5


for i in range(m2.num_cells()):
    if i in hcp_gids:
        peat_thickness[i] = 0.1 
        #irz_thickness[i] = hcp_irz
        irz_thickness[i] = irz_rand[i]
        #print ('HCP: ',i,peat_thickness[i])
    else:
        peat_thickness[i] = 0.2 #org_layer_bottom_bndry_gids(i)
        irz_thickness[i] = irz_rand[i] #irz_layer_bottom_bndry_gids(i,d_thick=0.56)
        #print ('LCP: ',i,peat_thickness[i])

XX = [0.02,0.04,0.08,0.12,0.16,0.2]
import itertools

def get_next(iterable):
    for item in itertools.cycle(iterable):
        yield item

pt = get_next(XX)
for i in range(m2.num_cells()):
    peat_thickness[i] = next(pt)
    #irz_thickness[i] = 0.5 #used in the true mesh
    irz_thickness[i] = irz_thickness[i] #using for manipulated test for NGEE 2021 talk
    
#print (peat_thickness, len(peat_thickness))
#print (irz_thickness)

In [ ]:
# Ice rich zone 40 cm for LCP, and 50 cm for HCP
#variable peat thickness
# layer extrusion
def make_mesh_variable_irz_om():
    layer_types = []
    layer_data = []
    layer_ncells = []
    layer_mat_ids = []

    z=0
    Z = []
    
    layer_cells = [10,10,20,20,19]
    layer_cells = [1,9,10,20,20,19]
    layer_flag =[False, True, True, False, False,False]
    
    Ncells = 0
    
    for i in range(layer_cells[0]):
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        if (layer_flag[0]):
            layer_mat_ids.append(-1001*np.ones((m2.num_cells(),),'i'))
        else:
            layer_mat_ids.append(1001*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
        Ncells +=1
        
        #print ('Moss done:', Ncells, Z[-1])
        
    for i in range(layer_cells[1]):
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        if (layer_flag[1]):
            layer_mat_ids.append(-1002*np.ones((m2.num_cells(),),'i'))
        else:
            layer_mat_ids.append(1002*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
        Ncells +=1
        
        #print ('Peat Done: ', Ncells, Z[-1])

    
    for i in range(layer_cells[2]): 
        layer_types.append('constant')
        layer_data.append(0.02)
        layer_ncells.append(1)
        if (layer_flag[2]):
            layer_mat_ids.append(-1003*np.ones((m2.num_cells(),),'i'))
        else:
            layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
        z = round(z + 0.02, 6)
        Z.append(z)
        Ncells +=1
        
        #print ('Ice rich zone 0: ', Ncells, Z[-1])

    dz = .02 #keep the top 60 cm to 2cm resolution
    
    for i in range(layer_cells[3]):
        dz = 0.02
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        if (layer_flag[3]):
            layer_mat_ids.append(-1004*np.ones((m2.num_cells(),),'i'))
        else:
            layer_mat_ids.append(1004*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
        Ncells +=1
        
        #print ('Ice rich zone 1: ', Ncells, Z[-1])
    
    for i in range(layer_cells[4]):
        dz *= 1.125
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        if (layer_flag[4]):
            layer_mat_ids.append(-1004*np.ones((m2.num_cells(),),'i'))
        else:
            layer_mat_ids.append(1004*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
        Ncells +=1
        #print ('Ice rich zone 2: ', Ncells, Z[-1])

    for i in range(layer_cells[4]):
        dz *= 1.2
        dz = np.round(dz,4)
        layer_types.append("constant")
        layer_data.append(dz)
        layer_ncells.append(1)
        layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))
        z = round(z + dz,6)
        Z.append(z)
        #print ('Mineral ',Z[-1])
    

    layer_types.append('snapped')
    layer_data.append(-45.0) # bottom location
    layer_ncells.append(1)
    layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))
    
    #---------------------------------------------------------------
    # Variable first layer: top = 1001
    mat_ids=np.zeros((m2.num_cells(), layer_cells[1]), 'i')
    for i in range(m2.num_cells()):
        for j in range(layer_cells[0],layer_cells[0]+layer_cells[1]):
            if (Z[j] <= peat_thickness[i]):
                mat_ids[i,j-layer_cells[0]]=1002     
            else:
                mat_ids[i,j-layer_cells[0]]=1003

    for j in range(layer_cells[0],layer_cells[0]+layer_cells[1]):
        layer_mat_ids[j] = mat_ids[:,j-layer_cells[0]]
        
        
    #----------------------------------------------------------------
    mat_ids=np.zeros((m2.num_cells(), layer_cells[2]), 'i')
    start=layer_cells[0] + layer_cells[1]
    end = start + layer_cells[2]
    #print (start, end)
    for i in range(m2.num_cells()):
        for j in range(start,end):
            #print ('IN: ',j, len(irz_thickness), len(Z))
            if (Z[j] >= irz_thickness[i]):
                mat_ids[i,j-start]=1004     
            else:
                mat_ids[i,j-start]=1003

    for j in range(start,end):
        layer_mat_ids[j] = mat_ids[:,j-start]

    
    m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                            layer_data, 
                                            layer_ncells, 
                                            layer_mat_ids)#,hcp_gids)

    #m3.write_exodus('barrow_polygon_watershed_5layer-var_irz5050_var_om_test.exo'.encode('utf8'))
    #m3.write_exodus('barrow_polygon_watershed_5layer-var_irz5050_var_om_movie1.exo'.encode('utf8'))
    s = 'barrow_watershed_5L-var_irz_var_om_movie2.exo'.encode(encoding='UTF-8')
    m3.write_exodus(s)

In [ ]:
#import imp
#imp.reload(meshing_ats)


In [ ]:
make_mesh_variable_irz_om()

In [ ]:
#write subgrid parameters to .h5 file
import h5py
infile = h5py.File('/Users/ajc/FUSE/simulations/production/old/polygonal_tundra.new/data_sg_para/sg_para-watershed468-new.h5','r')
outfile = h5py.File('/Users/ajc/FUSE/simulations/production/barrow_watershed/data/sg_parameters/sg_para-watershed468_1204.h5','w')
#hcp_gids
dmax = np.ones(468)*0.361
dex = np.ones(468)*0.361/2
delta = np.ones(468)*0.15
beta = np.ones(468)*7.0
for h in hcp_gids:
    dmax[h] = 0.416
    dex[h] = 0.228
    delta[h] = 0.08
    beta[h] = 1.5
for key in infile.keys():
    if not 'time' in key:
        d = infile[key]['0'][:1]
        grp = outfile.create_group(key)
        if 'microtopographic_relief' in key:
            grp.create_dataset('0',data=dmax)
        if 'excluded_volume' in key:
            grp.create_dataset('0',data=dex)
        if 'depression_depth' in key:
            grp.create_dataset('0',data=delta)
        if 'drag_exponent' in key:
            grp.create_dataset('0',data=beta)
    else:
        outfile.create_dataset('time',data=infile[key])
        #print (key,d[:4])
outfile.close()

In [ ]:
from scipy.spatial import ConvexHull
points = np.random.rand(4, 2)   # 30 random points in 2-D
hull = ConvexHull(points)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(points[:,0], points[:,1], 'o')
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-')